In [1]:
from ray.air import session, Checkpoint
import torch
from ray.train.huggingface import TransformersPredictor
import pandas as pd


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/opt/conda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /opt/conda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/opt/conda/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
cpoint=Checkpoint.from_directory("/domino/datasets/local/Ray_Deepspeed_GPTJ/TransformersTrainer_2023-07-28_19-53-53/TransformersTrainer_27302_00000_0_2023-07-28_19-53-53/checkpoint_000000")

In [3]:
cpoint.set_preprocessor(None)

In [4]:
torch.set_default_dtype(torch.float64)

prompts = pd.DataFrame(["Romeo and Juliet", "Romeo", "Juliet"], columns=["text"])
use_gpu = False

# Predict on the head node.
predictor = TransformersPredictor.from_checkpoint(
    checkpoint=cpoint,
    task="text-generation",
    torch_dtype=torch.float16 if use_gpu else None,
    device_map="auto",
    use_gpu=use_gpu,
)
prediction = predictor.predict(
    prompts,
    do_sample=True,
    temperature=0.9,
    min_length=32,
    max_length=128,
)

You have `use_gpu` as False but there are 4 GPUs detected on host where prediction will only use CPU. Please consider explicitly setting `TransformersPredictor(use_gpu=True)` or `batch_predictor.predict(ds, num_gpus_per_worker=1)` to enable GPU prediction. Ignore if you have set `device` or `device_map` arguments in the `pipeline` manually.
/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [5]:
prediction

,generated_text
0,Romeo and Juliet. Their parents' marriage. The...
1,"Romeo is in Verona, by the hypothesis, p. 414;..."
2,"Juliette's history, and its consequences, by t..."


In [8]:
prediction['generated_text'][0]

"Romeo and Juliet. Their parents' marriage. The betrothal of the respective heroines. Their ill-fated love. There are several other stories, likewise to be collected from ancient writers, which I do not now mention, but which I shall hereafter relate.\n their mutual correspondence and correspondence with others. The true account of the death of Romeo and his father. The meeting of the widow and the other daughters in the country. The marriage of the widowed sister to Count Paris. Her letter to Count Paris, with the letter to Lady Montague. The letter to Lady Montague, with the supposed letter of Lady"

In [6]:
prediction['generated_text'][1]

'Romeo is in Verona, by the hypothesis, p. 414; and by the way of the place (as above, p. 405) from thence to Milan, by a right line drawn to a given point O on the coast of the ocean, described at last by a given angle; and thence, by a right line drawn to a given point n on the coast of the Mediterranean, described at last by an angle; and then a right line drawn from the same point n to the point p, described by another angle, and thus the place of the point p is always given; the place of the point of departure'

In [7]:
prediction['generated_text'][2]

"Juliette's history, and its consequences, by the way, from the beginning of the year 1735 to the end of the year 1749, were written up in a very small manuscript by some of her relations, for the amusement of themselves and their families. The first sheet was sent to M. Clairville, at Paris, with a request that something might be inserted in the Gazette Litteraire, and was refused; the next to M. de la Boulie, at Aix, where it was also refused; and the last to M. de Croussey, at La Chaux, with the same reply"